In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

### Question 1

#### What to do ?

- Load the Energy Excel File
    - Specify the range
    - Change the column labels
    - Do some unit conversions
    - Cell values replacement
 
- Load the GDP csv
    - Specifiy the range
    - Replace cell values

- Load the ScimEn csv

- Join the 3 above dataset by Country, get top 15 according to the ScimEn dataset

In [3]:
def answer_one():
    import pandas as pd
    import numpy as np

    x = pd.ExcelFile('assets/Energy Indicators.xls')
    
    # Energy dataset ###################
    # specify the range + rename columns
    energy = x.parse(skiprows=17,skip_footer=(38))
    energy = energy[['Unnamed: 1','Petajoules','Gigajoules','%']]
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']] = energy[[
        'Energy Supply', 'Energy Supply per Capita', '% Renewable'
        ]].replace('...',np.NaN).apply(pd.to_numeric)
    
    # unit conversion
    energy['Energy Supply'] = energy['Energy Supply']*1000000
    
    # cell values replacement
    energy['Country'] = energy['Country'].replace(
        {'China, Hong Kong Special Administrative Region':'Hong Kong',
         'United Kingdom of Great Britain and Northern Ireland':'United Kingdom',
         'Republic of Korea':'South Korea',
         'United States of America':'United States',
         'Iran (Islamic Republic of)':'Iran'})
    energy['Country'] = energy['Country'].str.replace(r" \(.*\)","")
    
    # GDP ###############################
    #
    GDP = pd.read_csv('assets/world_bank.csv',skiprows=4)
    GDP['Country Name'] = GDP['Country Name'].replace('Korea, Rep.','South Korea')
    GDP['Country Name'] = GDP['Country Name'].replace('Iran, Islamic Rep.','Iran')
    GDP['Country Name'] = GDP['Country Name'].replace('Hong Kong SAR, China','Hong Kong')
    GDP = GDP[
        ['Country Name',
         '2006','2007','2008','2009','2010',
         '2011','2012','2013','2014','2015']
    ]
    GDP.columns = [
        'Country',
        '2006','2007','2008','2009','2010',
        '2011','2012','2013','2014','2015'
    ]
    
    # ScimEn ############################
    ScimEn = pd.read_excel(io='assets/scimagojr-3.xlsx')
    ScimEn_m = ScimEn[:15]
    
    df = pd.merge(ScimEn_m,energy,how='inner',left_on='Country',right_on='Country')
    final_df = pd.merge(df,GDP,how='inner',left_on='Country',right_on='Country')
    final_df = final_df.set_index('Country')
    
    return final_df

### Question 2

#### What to do ?

- calculate the diff between len(ScimEn) - len(ScimEn_m)

### Question 3
#### What to do ?
- top 15 countries for average GDP over 10 years (from 2006-2015)

In [2]:
def answer_three():
    
    top_15 = answer_one()
    columns = [str(x) for x in range(2006,2016)]
    avgGDP = top_15[columns]\
            .mean(axis=1).rename('avgGDP')\
            .sort_values(ascending=False)
    
    return avgGDP

### Question 4
#### What to do ?
- Simple arithmetic operations

In [5]:
def answer_four():
    
    top_15 = answer_one()
    answer = top_15[top_15['Rank'] == 4]['2015'] - top_15[top_15['Rank'] == 4]['2006']
    
    return answer.values[0]

### Question 5
#### What to do ?
- Simple arithmetic operations

In [6]:
def answer_five():
    top_15 = answer_one()
    return top_15['Energy Supply per Capita'].mean()

### Question 6
#### What to do ?
- find max value

In [7]:
def answer_six():
    
    top_15 = answer_one()
    max_ = top_15['% Renewable'].max()
    country_name = top_15[top_15['% Renewable'] == top_15['% Renewable'].max()].index[0]
    return (country_name, max_)

### Question 7
#### What to do ?
- perform axes calculations
- find max

In [8]:
def answer_seven():
    
    top_15 = answer_one()
    top_15['citation_ratio'] = top_15['Self-citations'] / top_15['Citations']
    max_ = top_15['citation_ratio'].max()
    country = top_15[top_15['citation_ratio'] == top_15['citation_ratio'].max()].index[0]
    
    return (country,value)

### Question 8
#### What to do ?
- perform axes calculations
- find and item based on ranking

In [9]:
def answer_eight():
    
    top_15 = answer_one()
    top_15.columns
    top_15['estimated_pop']= top_15['Energy Supply'] / top_15['Energy Supply per Capita']
    country = top_15['estimated_pop'].sort_values(ascending=False).index[2]
    
    return country

### Question 9
#### What to do ?
- perform axes calculations
- calculate correlations

In [10]:
def answer_nine():
    
    top_15 = answer_one()
    top_15['estimated_pop']= top_15['Energy Supply'] / top_15['Energy Supply per Capita']
    top_15['citable_docs_per_person'] = top_15['Citable documents'] / top_15['estimated_pop']   
    
    return top_15['Energy Supply per Capita'].corr(top_15['citable_docs_per_person'])

### Question 10
#### What to do ?
- perform axes calculations & create a column based on the result
- calculate correlations

In [11]:
def answer_ten():
    
    top_15 = answer_one()
    top_15['HighRenew'] = [1 if x >= top_15['% Renewable'].median() else 0 for x in top_15['% Renewable']]
    
    return top_15['HighRenew']

### Question 11
#### What to do ?
- create a new column
- perform aggregation function on grouped-by DF

In [13]:
def answer_eleven():
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    top_15 = answer_one()
    top_15['PopEst'] = (top_15['Energy Supply'] / top_15['Energy Supply per Capita']).astype(float)
    top_15 = top_15.reset_index()
    top_15['Continent'] = [ContinentDict[country] for country in top_15['Country']]
    
    answer = top_15.set_index('Continent').groupby(level=0)['PopEst'].agg({
        'size':np.size, 'sum':np.sum, 'mean':np.mean, 'std':np.std
    })
    answer = answer[['size','sum','mean','std']]
    
    return answer

### Question 12
#### What to do ?
- cut DF

In [15]:
def answer_twelve():
    
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
    top_15 = answer_one()
    top_15.reset_index()
    top_15['Continent'] = [ContinentDict[country] for country in top_15['Country']]
    top_15['bins'] = pd.cut(top_15['% Renewable'],5)
    x = top_15.groupby(['Continent','bins']).size()
    x.index.names = ['Continent','% Renewable']
    
    return x

### Question 13
#### What to do ?
- formatting

In [16]:
def answer_thirteen():
    
    top_15 = answer_one()
    top_15['PopEst'] =  (top_15['Energy Supply'] / top_15['Energy Supply per Capita']).astype(float)  
    
    top_15['PopEst'] = top_15['PopEst'].apply(lambda x: "{:,}".format(x))
    return top_15['PopEst']